for $\epsilon_z$ = 0.05

X       225.889093 \
Y       225.248591 \
Z      9691.726418 \
TXY      -1.524440 \
TYZ      -0.000099 \
TZX      -0.000075 \




In [8]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sqldf import sqldf
import pandas as pd

# Replace 'yourfile.h5' with the path to your HDF5 file
filepath = r'C:\\Users\\jonat\\Box\\PATRAN_NASTRAN\\rve_47_032224.h5'
filepath = r'C:\\Users\\jonat\Box\\PATRAN_NASTRAN\\run_zy_disp_0.05\\rve_47_zy_disp_0.05.h5'


# Populate dataframe for stress data

In [37]:
with h5py.File(filepath, 'r') as file:
    # Access the "TETRA" dataset
    stress_data = file['NASTRAN']['RESULT']['ELEMENTAL']['STRESS']['TETRA']
    # Read the dataset into a numpy array
    stress_array = stress_data[:]
    
    # Initialize an empty DataFrame
    stress_df = pd.DataFrame()
    
    # Iterate over the fields in the structured array and handle them accordingly
    for name in stress_array.dtype.names:
        # Special handling for the 'GRID' field
        if name == 'GRID':
            # Assuming 'GRID' is the name of the field with subarrays you want to split
            for i in range(stress_array[name].shape[1]):  # Iterate over the subarray length
                # Create new column names for each value in the GRID subarray
                new_col_name = f'{name}{i+1}'
                # Assign each value to its new column in the DataFrame
                stress_df[new_col_name] = stress_array[name][:, i]
        elif stress_array.dtype[name].shape:
            # For other fields that are subarrays, take only the first element
            stress_df[name] = stress_array[name][:, 0]
        else:
            # For basic types, directly assign to the DataFrame
            stress_df[name] = stress_array[name]


   EID  CID    CTYPE  NODEF  GRID1  GRID2  GRID3  GRID4  GRID5            X  \
0    1    0  b'GRID'      4      0    113    112      2    114   772.349772   
1    2    0  b'GRID'      4      0      3      1    114      2  2158.761123   
2    3    0  b'GRID'      4      0      1      2    112    114 -1172.982370   
3    4    0  b'GRID'      4      0      3    114      1      4  2158.761123   
4    5    0  b'GRID'      4      0    115      4    112    114   923.198882   

             Y             Z           TXY         TYZ         TZX  DOMAIN_ID  
0   421.233680  19138.280006  5.518242e+02 -300.696033 -746.230765          1  
1  2158.761123  19428.850105  6.088432e-13  -90.831033 -990.715044          1  
2  7177.001229  19628.962259 -6.717384e+02 -536.683460 -717.645873          1  
3  2158.761123  19428.850105  5.465473e-13  -90.831033 -990.715044          1  
4   853.114120  19164.694247  3.360298e+02 -226.422250 -833.428912          1  


# Populate dataframe for grid data

In [50]:
with h5py.File(filepath, 'r') as file:
    # Access the "TETRA" dataset
    grid_data = file['NASTRAN']['INPUT']['NODE']['GRID']
    # Read the dataset into a numpy array
    grid_array = grid_data[:]
    
    # Initialize an empty DataFrame
    grid_df = pd.DataFrame()
    
    for name in grid_array.dtype.names:
        if name == 'ID':
            grid_df[name] = grid_array[name]
        elif name == 'X':
            for i in range(grid_array[name].shape[1]):
                new_col_name = f'{name}{i+1}'
                grid_df[new_col_name] = grid_array[name][:,i]
                


   ID        X1        X2   X3
0   1  0.773509  0.773509  0.0
1   2  0.746158  0.746158  0.0
2   3  0.746480  0.750619  0.0
3   4  0.746924  0.755303  0.0
4   5  0.747459  0.760156  0.0


## Calculate the volume of each element using the grid information and grid ids

In [52]:
def calculate_volume(coords):
    A, B, C, D = coords
    AB, AC, AD = B - A, C - A, D - A
    volume = np.abs(np.dot(AB, np.cross(AC, AD))) / 6
    return volume

# Merge the grid coordinates into the elements dataframe
def merge_coordinates(df_elements, df_grid):
    # Merging each GRID column with df_grid to get coordinates
    merged_dfs = []
    for i in range(2, 6):
        merged_df = pd.merge(df_elements, df_grid, left_on=f'GRID{i}', right_on='ID', how='left')
        merged_df = merged_df.rename(columns={'X1': f'X{i}_1', 'X2': f'X{i}_2', 'X3': f'X{i}_3'})
        merged_dfs.append(merged_df[[f'X{i}_1', f'X{i}_2', f'X{i}_3']])
    
    # Concatenating the coordinate columns horizontally
    coords_df = pd.concat(merged_dfs, axis=1)
    
    # Calculating volumes and adding to df_elements
    volumes = coords_df.apply(lambda row: calculate_volume([np.array(row[['X2_1', 'X2_2', 'X2_3']]),
                                                            np.array(row[['X3_1', 'X3_2', 'X3_3']]),
                                                            np.array(row[['X4_1', 'X4_2', 'X4_3']]),
                                                            np.array(row[['X5_1', 'X5_2', 'X5_3']])]), axis=1)
    
    df_elements['Volume'] = volumes
    return df_elements

aggregate_df = merge_coordinates(stress_df, grid_df)

In [64]:
aggregate_df.head()

,EID,CID,CTYPE,NODEF,GRID1,GRID2,GRID3,GRID4,GRID5,X,Y,Z,TXY,TYZ,TZX,DOMAIN_ID,Volume
0,1,0,b'GRID',4,0,113,112,2,114,772.349772,421.233680,19138.280006,5.518242e+02,-300.696033,-746.230765,1,0.000002
1,2,0,b'GRID',4,0,3,1,114,2,2158.761123,2158.761123,19428.850105,6.088432e-13,-90.831033,-990.715044,1,0.000002
2,3,0,b'GRID',4,0,1,2,112,114,-1172.982370,7177.001229,19628.962259,-6.717384e+02,-536.683460,-717.645873,1,0.000002
3,4,0,b'GRID',4,0,3,114,1,4,2158.761123,2158.761123,19428.850105,5.465473e-13,-90.831033,-990.715044,1,0.000002
4,5,0,b'GRID',4,0,115,4,112,114,923.198882,853.114120,19164.694247,3.360298e+02,-226.422250,-833.428912,1,0.000002


In [66]:
# Multiply stress values by the volume
weighted_stresses = aggregate_df[['X', 'Y', 'Z', 'TXY', 'TYZ', 'TZX']].multiply(aggregate_df['Volume'], axis="index")

# Sum the weighted stresses and the total volume
total_weighted_stress = weighted_stresses.sum()
total_volume = aggregate_df['Volume'].sum()

# Divide the total weighted stresses by the total volume
average_stress = total_weighted_stress / total_volume

In [78]:
sigma = np.array([average_stress['X'], average_stress['Y'], average_stress['Z'], average_stress['TXY'], average_stress['TYZ'], average_stress['TZX']])  # Your strain data
epsilon = np.array([0, 0, 0.05, 0, 0, 0])  # Your stress data


In [83]:
E33 = sigma[2]/epsilon[2]

In [86]:
average_stress

X       225.889093
Y       225.248591
Z      9691.726418
TXY      -1.524440
TYZ      -0.000099
TZX      -0.000075
dtype: float64